In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import io
import os
import sys
from typing import (
    Dict, 
    List, 
    Any
)
import warnings

from tqdm import tqdm
from datetime import datetime

import numpy as np
import pandas as pd
from joblib import load, dump

from optbinning import OptimalBinning

import seaborn as sns
import matplotlib.pyplot as plt

(CVXPY) Jun 05 04:41:32 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jun 05 04:41:32 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.10.4067). Expected < 9.10.0. Please open a feature request on cvxpy to enable support for this version.')


In [3]:
sys.path.append(os.path.join(os.getcwd(), '..'))
from utils.metrics import Metrics

In [4]:
# set constants
COLUMNS_DICT: Dict[str, str] = {'OPTIN_11SCN_ML': 'Num', 
'G5_ND_SPECII': 'Num', 
'Marital_Status': 'Char', 
'SP_F2_33': 'Num', 
'AFF_B_09_01': 'Num', 
'TRD_A_20': 'Num', 
'SP_I_62_01': 'Num', 
'TRD_A_08': 'Num', 
'VM01_SP_VM2_15': 'Num', 
'GEN11_SP_N_92': 'Char', 
'SP_EDI_24': 'Num', 
'VM07_SP_VM1_11': 'Num', 
'ND_ECC_08': 'Num', 
'CLU_NPR_L1M': 'Num', 
'TRD_P_12': 'Num', 
'SP_H_41': 'Num', 
'G5_ND_SPACII': 'Num', 
'TRD_B_24': 'Num', 
'SP_I_61_01': 'Num', 
'TRD_O_16': 'Num', 
'SP_B2_21': 'Num', 
'TRD_P_11': 'Num', 
'VM01_SP_VM2_23': 'Num', 
'SP_G_37': 'Num', 
'E4_Q_17': 'Char', 
'TRD_C_07': 'Num', 
'TRD_C_03': 'Num', 
'TRD_B_20': 'Num', 
'SP_B2_18': 'Num', 
'TRD_B_22': 'Num', 
'SP_B3_23': 'Num', 
'TRD_P_10': 'Num', 
'SPA_F2_33': 'Num', 
'VM01_SP_VM2_14': 'Num', 
'TRD_B_19': 'Num', 
'VM01_SP_VM2_24': 'Num', 
'SP_G_38': 'Num', 
'TRD_O_05': 'Num', 
'SPA_B2_18': 'Num', 
'E1_A_11': 'Num', 
'E1_A_09': 'Num', 
'E1_A_10': 'Num', 
'PTSBR_L6M_NPR_L6M': 'Num', 
'E2_H_06': 'Num', 
'SP_F1_30': 'Num', 
'TRD_O_01': 'Num', 
'TRD_O_12': 'Num', 
'TRD_P_09': 'Num', 
'TRD_O_07': 'Num', 
'E1_A_05': 'Num', 
'ND_INC_03': 'Num', 
'VM01_SP_VM2_17': 'Num', 
'SP_B1_14': 'Num', 
'VM01_SP_VM2_18': 'Num', 
'GEN11_SP_N_91': 'Num', 
'GEN11_SP_EDI_07': 'Num', 
'AFF_T_66': 'Char', 
'PTBR_L3M_NPR_L3M': 'Num', 
'SP_EDI_03': 'Num', 
'ND_ECC_06': 'Num', 
'TRD_B_18': 'Num', 
'SPA_B2_19': 'Num', 
'SP_E1_28': 'Num', 
'SP_I_64_01': 'Num', 
'VM07_SP_VM1_18': 'Num', 
'NUM_DELINQ_ACCS': 'Num', 
'TRD_C_09': 'Num', 
'SP_F1_31': 'Num', 
'TRD_B_17': 'Num', 
'TRD_C_10': 'Num', 
'TRD_B_08': 'Num', 
'EA5_S_01': 'Num', 
'ND_HAC_04': 'Num', 
'VM01_SP_VM2_26': 'Num', 
'SPA_A_04': 'Num', 
'CLU_CLI_L6M_NPR_L6M': 'Num', 
'E1_B_06': 'Num', 
'TRD_A_04': 'Num', 
'SP_I_67_01': 'Char', 
'GEN11_SP_K_80': 'Char', 
'TRD_B_07': 'Num', 
'TRD_A_13': 'Num', 
'SP_I_63_01': 'Num', 
'SP_B2_20': 'Num', 
'E4_Q_04': 'Num', 
'TRD_RAG_01': 'Num', 
'TRD_A_03': 'Num', 
'VM09_SP_VM1_19': 'Num', 
'ND_ECC_04': 'Num', 
'VM08_SP_VM2_15': 'Num', 
'SP_B1_13': 'Num', 
'VAL_DELINQ_ACCS': 'Num', 
'AGE_MOST_RECENT_DELINQ': 'Num', 
'E1_A_03': 'Num', 
'E1_A_02': 'Num', 
'ND_ERL_01': 'Num', 
'VM08_SP_VM2_20': 'Num', 
'EA1_D_02': 'Num', 
'SP_I_56_01': 'Num', 
'ND_HAC_06': 'Num', 
'AFF_E_07_01': 'Num', 
'TRD_B_02': 'Num', 
'SP_EDI_17': 'Char', 
'TRD_B_36': 'Num', 
'TRD_O_06': 'Num', 
'SP_EDI_26': 'Num', 
'VM02_SP_VM1_16': 'Num', 
'TRD_B_05': 'Num', 
'E2_G_08': 'Num', 
'AFF_T_64_01': 'Char', 
'SP_EDI_15': 'Char', 
'TRD_B_32': 'Num', 
'SP_EDI_27': 'Num', 
'NUM_SETTLED_ACCS': 'Num', 
'TRD_B_06': 'Num', 
'VM08_SP_VM2_16': 'Num', 
'SP_EDI_37': 'Num', 
'VM04_SP_VM1_19': 'Num', 
'E2_H_04': 'Num', 
'TRD_B_52': 'Num', 
'SP_B1_11': 'Num', 
'AGE_MOST_RECENT_SETTD': 'Num', 
'AFF_E_29': 'Char', 
'NUM_INACTIVE_ACCS': 'Num', 
'NPD_NUM_NPD_SP_L18M': 'Num', 
'VM09_SP_VM1_04': 'Char', 
'AFF_T_07_01': 'Num', 
'TRD_B_53': 'Num', 
'NPD_NUM_EBAD_SP_L18M': 'Num', 
'VM03_SP_VM2_27': 'Num', 
'VM02_SP_VM1_18': 'Num', 
'SPA_B3_22': 'Num', 
'VM04_SP_VM1_21': 'Num', 
'VM04_SP_VM1_20': 'Num', 
'NPD_BAL_NPD_SP_L18M': 'Num', 
'SP_EDI_30': 'Num', 
'VM03_SP_VM2_28': 'Num', 
'TRD_A_17': 'Num', 
'VM10_SP_VM2_15': 'Num', 
'TRD_B_30': 'Num', 
'AFF_T_43': 'Num', 
'TRD_B_31': 'Num', 
'VM05_SP_VM1_19': 'Num', 
'AGE_MOST_RECENT': 'Num', 
'AFF_T_34_01': 'Num', 
'VM08_SP_VM2_14': 'Num', 
'TRD_C_05': 'Num', 
'ND_HAC_02': 'Num', 
'NDMAICH': 'Char', 
'VM09_SP_VM1_18': 'Num', 
'TRD_B_29': 'Num', 
'SP_J_61_01': 'Num', 
'SP_J_58_01': 'Num', 
'NPD_BAL_EBAD_SP_L18M': 'Num', 
'E1_B_08': 'Char', 
'VM08_SP_VM2_27': 'Num', 
'SP_EDI_22': 'Num', 
'VM05_SP_VM1_16': 'Num', 
'SP_EDI_31': 'Num', 
'VM10_SP_VM2_22': 'Num', 
'VM08_SP_VM2_18': 'Num', 
'TRD_A_21': 'Num', 
'TRD_A_18': 'Num', 
'VM08_SP_VM2_23': 'Num', 
'VM10_SP_VM2_27': 'Num', 
'VM08_SP_VM2_28': 'Num', 
'NUM_INACTIVE_ACCS_L36M': 'Num', 
'SPA_F3_36': 'Num', 
'E1_E_02': 'Num', 
'VM10_SP_VM2_23': 'Num', 
'SP_EDI_29': 'Num', 
'ND_HAC_03': 'Num', 
'E1_A_08': 'Num', 
'VM03_SP_VM2_24': 'Num', 
'TRD_STL_19': 'Num', 
'VM07_SP_VM1_26': 'Num', 
'VM04_SP_VM1_03': 'Char', 
'EA4_P_01': 'Char', 
'TRD_A_12': 'Num', 
'SP_I_66_01': 'Char', 
'TRD_STL_20': 'Num', 
'VM08_SP_VM2_07': 'Num', 
'VM01_SP_VM2_28': 'Num', 
'VM01_SP_VM2_29': 'Num', 
'VM04_SP_VM1_04': 'Char', 
'TRD_P_23': 'Num', 
'AFF_S_22_01': 'Char', 
'VM08_SP_VM2_17': 'Num', 
'AFF_C_06': 'Char', 
'GEN11_SP_J_58': 'Num', 
'E2_K_02': 'Num', 
'SP_B2_19': 'Num', 
'SP_EDI_12': 'Char', 
'VM03_SP_VM2_16': 'Num', 
'SP_EDI_19': 'Char', 
'VM10_SP_VM2_28': 'Num', 
'VM10_SP_VM2_29': 'Num', 
'AFF_V_19': 'Num', 
'SPA_A_01': 'Num', 
'AFF_S_03_01': 'Char', 
'VM04_SP_VM1_15': 'Num', 
'E1_B_13': 'Num', 
'VM08_SP_VM2_03': 'Char', 
'VM10_SP_VM2_16': 'Num', 
'VM04_SP_VM1_11': 'Num', 
'TRD_P_36': 'Num', 
'EA5_U_01': 'Num', 
'VM02_SP_VM1_05': 'Char', 
'GEN11_SP_L_81': 'Num', 
'HC_C_01': 'Num', 
'AFF_C_03_01': 'Char', 
'VM02_SP_VM1_11': 'Num', 
'VM01_SP_VM2_34': 'Num', 
'TRD_A_16': 'Num', 
'E2_J_02': 'Num', 
'VM01_SP_VM2_32': 'Num', 
'VM03_SP_VM2_07': 'Num', 
'VM08_SP_VM2_34': 'Num', 
'E2_H_01': 'Num', 
'VM01_SP_VM2_02': 'Char', 
'ND_ECC_05': 'Num', 
'AFF_M_01_01': 'Num', 
'TRD_O_08': 'Num', 
'VM05_SP_VM1_20': 'Num', 
'VM05_SP_VM1_21': 'Num', 
'NPD_NUM_NPD_SP_L12M': 'Num', 
'VM04_SP_VM1_26': 'Num', 
'NPD_NUM_EBAD_SP_L12M': 'Num', 
'PD_B_06': 'Num', 
'E2_H_02': 'Num', 
'VM04_SP_VM1_22': 'Num', 
'VM04_SP_VM1_16': 'Num', 
'TRD_A_23': 'Num', 
'AFF_T_82_01': 'Char', 
'VM04_SP_VM1_18': 'Num', 
'E1_B_12': 'Num', 
'VM09_SP_VM1_20': 'Num', 
'VM09_SP_VM1_21': 'Num', 
'TRD_RAG_13_AIQ2': 'Char', 
'VM08_SP_VM2_19': 'Num', 
'ND_PSD_05': 'Num', 
'SP_B1_15': 'Num', 
'TRD_A_19': 'Num', 
'EA2_Q_02': 'Num', 
'NPD_BAL_NPD_SP_L12M': 'Num', 
'ND_HAC_10': 'Num', 
'VM04_SP_VM1_01': 'Num', 
'GEN11_SP_L_78': 'Num', 
'SP_EDI_32': 'Num', 
'VM05_SP_VM1_04': 'Char', 
'AFF_V_09': 'Num', 
'ND_HAC_05': 'Num', 
'VM04_SP_VM1_08': 'Num', 
'NDMAUNSECARR': 'Char', 
'ND_ECC_01': 'Num', 
'E2_K_01': 'Num', 
'NO_CA_L3M': 'Num', 
'ND_ECC_10': 'Num', 
'VM04_SP_VM1_07': 'Num', 
'HC_B_01': 'Num', 
'E2_H_11': 'Num', 
'SPA_H_41': 'Num', 
'E2_G_09': 'Num', 
'ND_PSD_11': 'Num', 
'TRD_P_08': 'Num', 
'SP_J_60_01': 'Num', 
'SP_J_56_01': 'Num', 
'PD_F_21': 'Num', 
'VM04_SP_VM1_06': 'Num', 
'E1_B_04': 'Num', 
'SPA_E1_26': 'Num', 
'SP_C_24': 'Char', 
'VM10_SP_VM2_18': 'Num', 
'VM10_SP_VM2_30': 'Num', 
'NPD_BAL_EBAD_SP_L12M': 'Num', 
'AFF_V_17_01': 'Num', 
'AFF_T_56_01': 'Num', 
'SP_EDI_38': 'Num', 
'TRD_B_49': 'Num', 
'VM07_SP_VM1_10': 'Num', 
'E2_G_01': 'Num', 
'AFF_T_67_01': 'Char', 
'AFF_C_19_01': 'Char', 
'VM06_SP_VM1_05': 'Char', 
'NPD_NUM_NPD_SPA_L18M': 'Num', 
'NPD_NUM_EBAD_SPA_L18M': 'Num', 
'AFF_V_13_01': 'Num', 
'VM10_SP_VM2_07': 'Num', 
'TRD_RAG_13': 'Char', 
'ND_LNK_01': 'Char', 
'ND_ECC_03': 'Num', 
'VM01_SP_VM2_12': 'Num', 
'EA4_Q_06': 'Char', 
'VM03_SP_VM2_26': 'Num'} 

MIN_IV: float = 0.1
CORR_PEARSON_CUT_OFF: float = 0.75
RANDOM_SEED: int = 42


TARGET_NAME: str = 'GB6_Flag_2Limit'
PALETTE_TARGET: Dict[str, str] = {
    'G': 'green',
    'B': 'red',
    'I': 'black'
}
TARGET_BIN: str = f'_{TARGET_NAME}_bin'
TARGET_BIN_IisB: str = f'_{TARGET_NAME}_bin_IisB'

PROJECT_PATH: str = os.path.join(os.getcwd(), '..')
DATA_PATH: str = os.path.join(PROJECT_PATH, 'data')
OPT_BIN_PATH: str = os.path.join(os.path.join(PROJECT_PATH, 'objects'), 'optbinners')

In [5]:
# read data
df: pd.DataFrame = pd.read_csv(os.path.join(DATA_PATH, 'final_aiq2.csv'))

C:\Users\hdychko\AppData\Local\Temp\ipykernel_24100\1622656214.py:2: DtypeWarning: Columns (310,1307,1349,1351,1352,1400,1401,1402,1403,1404,1405) have mixed types. Specify dtype option on import or set low_memory=False.
  df: pd.DataFrame = pd.read_csv(os.path.join(DATA_PATH, 'final_aiq2.csv'))


In [6]:
print('Number of rows: %.0f; columns: %.0f' % df.shape)

Number of rows: 110462; columns: 1774


In [7]:
#  check if all necessary columns are present
assert df[COLUMNS_DICT.keys()].shape[1] == len(COLUMNS_DICT)

In [8]:
TRAIN_BIN_FROM: datetime.date = pd.to_datetime('2021-07-01').date()
TRAIN_BIN_TO: datetime.date = pd.to_datetime('2021-12-01').date()

# Preprocessing

## Create the target 'I', 'B' are 1, 'G' - 0

In [9]:
warnings.filterwarnings("ignore", category=FutureWarning)
df[TARGET_BIN_IisB] = df[TARGET_NAME].replace(
    {
        'G': 0,
        'B': 1, 
        'I': 1
    }
)

In [10]:
# check the distribution
df.groupby(TARGET_NAME)[TARGET_BIN_IisB].value_counts()

GB6_Flag_2Limit  _GB6_Flag_2Limit_bin_IisB
B                1                             4449
G                0                            99444
I                1                             6569
Name: count, dtype: int64

In [11]:
# compare with already existed column
df.groupby(TARGET_NAME)[TARGET_BIN].value_counts()

GB6_Flag_2Limit  _GB6_Flag_2Limit_bin
B                1                        4449
G                0                       99444
I                0                        6569
Name: count, dtype: int64

## Convert dates to the corresponding types

In [12]:
for col in ['_RDATE_EOM', '_RDATE']:
    df[col] = pd.to_datetime(df[col]).dt.date

In [13]:
print('Available date range:')
df['_RDATE'].agg(['min', 'max'])

Available date range:


min    2021-06-30
max    2023-11-30
Name: _RDATE, dtype: object

## Dafine dataset to train OptBinnning on

In [14]:
df_train: pd.DataFrame = df[
    (df._RDATE >= TRAIN_BIN_FROM) &
    (df._RDATE < TRAIN_BIN_TO) & 
    (df[TARGET_NAME] != 'I')
]

In [15]:
print('Number of rows: %.0f; columns: %.0f' % df_train.shape)

Number of rows: 13940; columns: 1775


In [16]:
print('Target population of the selected period:')
df_train.groupby(['_RDATE_EOM'])[TARGET_NAME].value_counts(sort=False).sort_index().rename('n')

Target population of the selected period:


_RDATE_EOM  GB6_Flag_2Limit
2021-07-31  B                    65
            G                  2348
2021-08-31  B                    66
            G                  2383
2021-09-30  B                    76
            G                  2657
2021-10-31  B                   131
            G                  3054
2021-11-30  B                    93
            G                  3067
Name: n, dtype: int64

In [17]:
df_train[TARGET_BIN_IisB].value_counts()

_GB6_Flag_2Limit_bin_IisB
0    13509
1      431
Name: count, dtype: int64

# Binning

## Numeric columns

In [18]:
exceptions = []
for col_name, col_type in tqdm(COLUMNS_DICT.items()):
    try: 
        unique_vals = df[col_name].unique()
        special_cases: List[Any] = []
        dict_to_replace: Dict[str, float] = dict()
        unique_floats: List[float] = []
        for elem in unique_vals:
            try:
                dict_to_replace[elem] = float(elem)
                unique_floats += [float(elem)]
            except ValueError:
                special_cases += [elem] 
        
        unique_floats = np.array(unique_floats) 
        special_cases += list(unique_floats[unique_floats < 0])
        if_numeric = col_type.lower().strip() == 'num'
        if len(special_cases) > 0:
            special_cases = dict(zip(special_cases, special_cases))
        
        if if_numeric:
            float(elem) 
            optb = OptimalBinning(
                name=col_name, 
                dtype='numerical', 
                solver="mip",
                special_codes=special_cases if len(special_cases) > 0 else None,
                monotonic_trend='auto_asc_desc' 
            )
            optb.fit(df_train[col_name].replace(dict_to_replace).values, df_train[TARGET_BIN_IisB].values)
            df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
            dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-1.jblb'))
            optbins_params = optb.get_params()
            dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-1.jblb'))
    except Exception as e:
        exceptions += [(col_name, e)]

 34%|███████████████████████████▎                                                     | 99/294 [01:46<01:00,  3.24it/s]C:\Users\hdychko\AppData\Local\Temp\ipykernel_24100\1834109457.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
 34%|███████████████████████████▍                                                    | 101/294 [01:55<05:59,  1.86s/it]C:\Users\hdychko\AppData\Local\Temp\ipykernel_24100\1834109457.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fr

In [19]:
print('If any Exception: ', len(exceptions))

If any Exception:  2


In [20]:
print('Number of rows: %.0f; columns: %.0f' % df.shape)

Number of rows: 110462; columns: 2030


In [21]:
# list columns which weren't transformed
if exceptions != []:
    exceptions_cols = [elem[0] for elem in exceptions]
    print(exceptions_cols)

['SP_EDI_37', 'SP_EDI_38']


In [22]:
for col in exceptions_cols:
    print(col, ': ', COLUMNS_DICT[col].lower().strip())
    print('Unique: ', df[col].unique())
    print('-'*50)

SP_EDI_37 :  num
Unique:  ['D' 'C' 'H' 'B' 'G' 'X' 'E' 'F' 'A' 'T' 'I']
--------------------------------------------------
SP_EDI_38 :  num
Unique:  ['B' 'X' 'T' 'G' 'E' 'D']
--------------------------------------------------


In [23]:
# Columns with only categorical values are defined as Num. The default `Num` type is changed to `Char`
for col in exceptions_cols:
    COLUMNS_DICT[col] = 'Char'

## Categorical columns

In [24]:
exceptions_cat = []
for col_name, col_type in tqdm(COLUMNS_DICT.items()):
    try: 
        unique_vals = df[col_name].unique()
        special_cases: List[Any] = []
        dict_to_replace: Dict[str, float] = dict()
        unique_floats: List[float] = []
        for elem in unique_vals:
            try:
                dict_to_replace[elem] = float(elem)
                unique_floats += [float(elem)]
            except ValueError:
                pass

        unique_floats = np.array(unique_floats)
        special_cases += list(unique_floats[unique_floats < 0])

        if len(special_cases) > 0:
            special_cases = dict(zip(special_cases, special_cases))
        
        if_numeric = col_type.lower().strip() == 'num'
        if not if_numeric:
            optb = OptimalBinning(
                name=col_name, 
                dtype='categorical', 
                solver="mip",
                special_codes=special_cases if len(special_cases) > 0 else None
            )
            optb.fit(df_train[col_name].replace(dict_to_replace).values, df_train[TARGET_BIN_IisB].values)
            df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
            dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-1.jblb'))
            optbins_params = optb.get_params()
            dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-1.jblb'))
    except Exception as e:
        exceptions_cat += [(col_name, e)]

  0%|                                                                                          | 0/294 [00:00<?, ?it/s]C:\Users\hdychko\AppData\Local\Temp\ipykernel_24100\3748283012.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
C:\Users\hdychko\AppData\Local\Temp\ipykernel_24100\3748283012.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).val

In [25]:
print('If any Exception: ', len(exceptions_cat))

If any Exception:  0


In [26]:
print('Number of rows: %.0f; columns: %.0f' % df.shape)

Number of rows: 110462; columns: 2069


## Modification of splits of several columns

### TRD_O_07

In [27]:
df[df.TRD_O_07 < -3].TRD_O_07.value_counts()

TRD_O_07
-4     3
-13    3
-10    1
-8     1
-11    1
-9     1
Name: count, dtype: int64

In [28]:
df[df.TRD_O_07 < -3].groupby('TRD_O_07')._RDATE_EOM.unique().sort_index()

TRD_O_07
-13    [2022-03-31, 2022-09-30, 2022-08-31]
-11                            [2022-11-30]
-10                            [2022-12-31]
-9                             [2022-02-28]
-8                             [2022-08-31]
-4     [2022-12-31, 2023-06-30, 2022-03-31]
Name: _RDATE_EOM, dtype: object

In [29]:
COLUMNS_SP_SPLITS: Dict[str, Any] = {
    'TRD_B_24': dict(
        zip(
            [
                -999999,
                -999998,
                -999997,
                -999996,
                -999995,
            ], 
            [
                -999999,
                -999998,
                -999997,
                -999996,
                -999995,
            ]
        )
    ),
    'TRD_B_18': dict(
        zip(
            [
                -999999,
                -999998,
                -999997,
                -999996,
                -999995,
                -999994,
            ],
            [
                -999999,
                -999998,
                -999997,
                -999996,
                -999995,
                -999994,
            ]
        )
    ),
    'TRD_B_08': dict(
        zip(
            [
                -999999,
                -999998,
                -999997,
                -999996,
                -999995,
                -999994,
            ],
            [
                -999999,
                -999998,
                -999997,
                -999996,
                -999995,
                -999994,
            ]
        )
    )
}

In [30]:
exceptions = []
for col_name in tqdm(COLUMNS_SP_SPLITS.keys()):
    # if col_name == 'TRD_B_24':
        try: 
            unique_vals = df[col_name].unique()
            special_cases: List[Any] = []
            dict_to_replace: Dict[str, float] = dict()
            unique_floats: List[float] = []
            for elem in unique_vals:
                try:
                    dict_to_replace[elem] = float(elem)
                    unique_floats += [float(elem)]
                except ValueError:
                    special_cases += [elem] 
    
            if len(special_cases) > 0:
                raise ValueError()
    
            float(elem) 
            optb = OptimalBinning(
                name=col_name, 
                dtype='numerical', 
                solver="mip",
                special_codes=COLUMNS_SP_SPLITS[col_name],
                monotonic_trend='auto_asc_desc' 
            )
            optb.fit(df_train[col_name].replace(dict_to_replace).values, df_train[TARGET_BIN_IisB].values)
            df[col_name + 'OptBin'] = optb.transform(df[col_name].replace(dict_to_replace).values, metric="woe")
            dump(optb, os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-1.jblb'))
            optbins_params = optb.get_params()
            print(optbins_params)
            dump(optbins_params, os.path.join(OPT_BIN_PATH, f'[params]{col_name}-negatives-separately-I-1.jblb'))
            temp = optb.binning_table.build()
        except Exception as e:
            exceptions += [(col_name, e)]

 33%|████████████████████████████                                                        | 1/3 [00:01<00:02,  1.08s/it]

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': None, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'TRD_B_24', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-999999: -999999, -999998: -999998, -999997: -999997, -999996: -999996, -999995: -999995}, 'split_digits': None, 'time_limit': 100, 'user_splits': None, 'user_splits_fixed': None, 'verbose': False}


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:03<00:01,  1.77s/it]

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': None, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'TRD_B_18', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-999999: -999999, -999998: -999998, -999997: -999997, -999996: -999996, -999995: -999995, -999994: -999994}, 'split_digits': None, 'time_limit': 100, 'user_splits': None, 'user_splits_fixed': None, 'verbose': False}


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.34s/it]

{'cat_cutoff': None, 'cat_unknown': None, 'class_weight': None, 'divergence': 'iv', 'dtype': 'numerical', 'gamma': 0, 'max_bin_n_event': None, 'max_bin_n_nonevent': None, 'max_bin_size': None, 'max_n_bins': None, 'max_n_prebins': 20, 'max_pvalue': None, 'max_pvalue_policy': 'consecutive', 'min_bin_n_event': None, 'min_bin_n_nonevent': None, 'min_bin_size': None, 'min_event_rate_diff': 0, 'min_n_bins': None, 'min_prebin_size': 0.05, 'mip_solver': 'bop', 'monotonic_trend': 'auto_asc_desc', 'name': 'TRD_B_08', 'outlier_detector': None, 'outlier_params': None, 'prebinning_method': 'cart', 'solver': 'mip', 'special_codes': {-999999: -999999, -999998: -999998, -999997: -999997, -999996: -999996, -999995: -999995, -999994: -999994}, 'split_digits': None, 'time_limit': 100, 'user_splits': None, 'user_splits_fixed': None, 'verbose': False}


In [31]:
print('If any Exception: ', len(exceptions))

If any Exception:  0


# WoE per bin

In [33]:
def find_max_of_negative_bins(x):
    v: Union[float, np.nan] = np.nan
    try: 
        v = float(x)
    except:
        pass
    return v

In [34]:
#  extract intervals
warnings.filterwarnings("ignore", category=FutureWarning)
df_intervals = pd.DataFrame()
for col_name in COLUMNS_DICT.keys():
    optb = load(os.path.join(OPT_BIN_PATH, f'{col_name}-negatives-separately-I-1.jblb'))
    temp = optb.binning_table.build()
    temp['Feature'] = col_name
    temp.Bin = temp.Bin.astype(str)
    temp = temp[~((temp.Bin == 'Missing') & (temp.Count == 0))]
    temp = temp[(temp.Bin != '')]
    temp = temp.reset_index(drop=True)
    df_intervals = pd.concat((df_intervals, temp))
    
temp = df_intervals.groupby('Feature').apply(lambda y: y.Bin.apply(lambda x: find_max_of_negative_bins(x)).max()).reset_index(name='min_val')
df_intervals = pd.merge(df_intervals, temp, on='Feature', how='left')
df_intervals['Bin'] = df_intervals.apply(lambda x: x.Bin.replace('(-inf', '(' + str(x.min_val)) if x.min_val is not np.nan else x.Bin, axis=1)
del df_intervals['min_val']

C:\Users\hdychko\AppData\Local\Temp\ipykernel_24100\262799576.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = df_intervals.groupby('Feature').apply(lambda y: y.Bin.apply(lambda x: find_max_of_negative_bins(x)).max()).reset_index(name='min_val')


In [35]:
df_intervals

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature
0,"(-997.0, 684.50)",1327,0.095194,1145,182,0.137151,-1.60585,0.541999,0.061296,OPTIN_11SCN_ML
1,"[684.50, 760.50)",1100,0.078910,1001,99,0.090000,-1.131368,0.176040,0.020902,OPTIN_11SCN_ML
2,"[760.50, 814.50)",845,0.060617,796,49,0.057988,-0.657224,0.035993,0.004420,OPTIN_11SCN_ML
3,"[814.50, 859.50)",699,0.050143,676,23,0.032904,-0.064304,0.000214,0.000027,OPTIN_11SCN_ML
4,"[859.50, 955.50)",1661,0.119154,1627,34,0.020470,0.423129,0.017582,0.002181,OPTIN_11SCN_ML
...,...,...,...,...,...,...,...,...,...,...
16245,['Y'],911,0.065352,868,43,0.047201,-0.440012,0.015627,0.001938,EA4_Q_06
16246,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000,EA4_Q_06
16247,"(-1.0, inf)",3330,0.238881,3179,151,0.045345,-0.397961,0.045775,0.005684,VM03_SP_VM2_26
16248,-2.0,10569,0.758178,10289,280,0.026493,0.159038,0.017810,0.002224,VM03_SP_VM2_26


## Check

In [36]:
np.random.seed(RANDOM_SEED)
random_col = np.random.choice(list(COLUMNS_DICT.keys()), 1)[0]
df_intervals[df_intervals.Feature == random_col]

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature
13334,[ 49. 236. 238. 239. 240. 241. 242. 243. 244. ...,2892,0.207461,2888,4,0.001383,3.137022,0.641529,0.058012,SP_EDI_17
13335,[160. 128. 147. 167. 134. 122.],766,0.054950,753,13,0.016971,0.614113,0.015708,0.001933,SP_EDI_17
13336,[149. 151. 104. 141. 123. 130. 181.],841,0.060330,822,19,0.022592,0.322298,0.005403,0.000672,SP_EDI_17
13337,[148. 170. 162. 204. 143. 103. 177.],778,0.055811,757,21,0.026992,0.139837,0.001023,0.000128,SP_EDI_17
13338,[142. 163. 145. 210. 191. 156. 153. 124. 132. ...,1313,0.094189,1274,39,0.029703,0.041352,0.000158,0.000020,SP_EDI_17
13339,[183. 158. 155. 131. 168. 217. 138. 202. 127. ...,1106,0.079340,1069,37,0.033454,-0.081442,0.000547,0.000068,SP_EDI_17
13340,[237. 137. 139. 144. 172. 164. 146. 249. 247. ...,1071,0.076829,1029,42,0.039216,-0.24633,0.005241,0.000653,SP_EDI_17
13341,[185. 118. 150. 208. 105. 159. 152. 100. 129.],916,0.065710,876,40,0.043668,-0.358517,0.010025,0.001246,SP_EDI_17
13342,[120. 113. 111. 135. 184. 211. 214. 112. 99. ...,828,0.059397,787,41,0.049517,-0.490347,0.018079,0.002238,SP_EDI_17
13343,[115. 136. 199. 91. 108. 126. 116. 203. 197. ...,824,0.059110,777,47,0.057039,-0.639711,0.032965,0.004052,SP_EDI_17


In [37]:
df_train[df_train[random_col] == -2].shape

(1793, 1775)

In [38]:
df_train[df_train[random_col] == -2][TARGET_NAME].value_counts()

GB6_Flag_2Limit
G    1735
B      58
Name: count, dtype: int64

In [39]:
df_train[df_train[random_col] == -2][TARGET_BIN_IisB].value_counts()

_GB6_Flag_2Limit_bin_IisB
0    1735
1      58
Name: count, dtype: int64

In [40]:
df_intervals.to_csv('bins_optbins_neg_sep_I_bad.csv')

In [41]:
COLS_TO_SAVE = [
'OPTIN_11SCN_MLOptBin',
'G5_ND_SPECIIOptBin',
'Marital_StatusOptBin',
'SP_F2_33OptBin',
'AFF_B_09_01OptBin',
'TRD_A_20OptBin',
'TRD_A_08OptBin',
'VM01_SP_VM2_15OptBin',
'GEN11_SP_N_92OptBin',
'SP_EDI_24OptBin',
'SP_G_37OptBin',
'E4_Q_17OptBin',
'TRD_C_07OptBin',
'TRD_B_20OptBin',
'SP_B2_18OptBin',
'SPA_F2_33OptBin',
'VM01_SP_VM2_14OptBin',
'VM01_SP_VM2_24OptBin',
'SP_G_38OptBin',
'TRD_O_05OptBin',
'E1_A_09OptBin',
'TRD_O_07OptBin',
'ND_INC_03OptBin',
'VM01_SP_VM2_17OptBin',
'SP_B1_14OptBin',
'GEN11_SP_N_91OptBin',
'GEN11_SP_EDI_07OptBin',
'ND_ECC_06OptBin',
'TRD_B_18OptBin',
'TRD_B_08OptBin',
'EA5_S_01OptBin',
'CLU_CLI_L6M_NPR_L6MOptBin',
'E1_B_06OptBin',
'SP_I_67_01OptBin',
'GEN11_SP_K_80OptBin',
'SP_I_63_01OptBin',
'SP_B2_20OptBin',
'E4_Q_04OptBin',
'ND_ECC_04OptBin',
'VM08_SP_VM2_15OptBin',
'VAL_DELINQ_ACCSOptBin',
'ND_ERL_01OptBin',
'VM08_SP_VM2_20OptBin',
'EA1_D_02OptBin',
'ND_HAC_06OptBin',
'AFF_E_07_01OptBin',
'TRD_O_06OptBin',
'E2_G_08OptBin',
'AFF_T_07_01OptBin',
'VM02_SP_VM1_18OptBin'
]
df_intervals[df_intervals.Feature.isin([col[:-len('OptBin')] for col in COLS_TO_SAVE])].to_csv('bins_optbins_neg_sep_I_bad_sample.csv', index=None)

In [42]:
df_intervals

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature
0,"(-997.0, 684.50)",1327,0.095194,1145,182,0.137151,-1.60585,0.541999,0.061296,OPTIN_11SCN_ML
1,"[684.50, 760.50)",1100,0.078910,1001,99,0.090000,-1.131368,0.176040,0.020902,OPTIN_11SCN_ML
2,"[760.50, 814.50)",845,0.060617,796,49,0.057988,-0.657224,0.035993,0.004420,OPTIN_11SCN_ML
3,"[814.50, 859.50)",699,0.050143,676,23,0.032904,-0.064304,0.000214,0.000027,OPTIN_11SCN_ML
4,"[859.50, 955.50)",1661,0.119154,1627,34,0.020470,0.423129,0.017582,0.002181,OPTIN_11SCN_ML
...,...,...,...,...,...,...,...,...,...,...
16245,['Y'],911,0.065352,868,43,0.047201,-0.440012,0.015627,0.001938,EA4_Q_06
16246,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000,EA4_Q_06
16247,"(-1.0, inf)",3330,0.238881,3179,151,0.045345,-0.397961,0.045775,0.005684,VM03_SP_VM2_26
16248,-2.0,10569,0.758178,10289,280,0.026493,0.159038,0.017810,0.002224,VM03_SP_VM2_26


### Compute IV

In [43]:
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

cat_failed_cols = []
df_iv = pd.DataFrame()
for col in tqdm([elem + 'OptBin' for elem in COLUMNS_DICT.keys()]):
    try:
        m = Metrics()
        df_iv_temp = m.iv_categorical(
            df_actual=df,s
            feature_col=col,
            target_col=TARGET_BIN,
            positive_inf_offset=0.003,
            negative_inf_offset=0.003,
        )[0]
        df_iv = pd.concat((df_iv, df_iv_temp))
    except Exception as e:
        cat_failed_cols += [(col, e)]

100%|████████████████████████████████████████████████████████████████████████████████| 294/294 [00:06<00:00, 44.39it/s]


In [44]:
print('If any Exceptions: ', len(cat_failed_cols))

If any Exceptions:  0


In [45]:
df_iv.head()

,Feature,IV
0,OPTIN_11SCN_MLOptBin,1.429000
0,G5_ND_SPECIIOptBin,1.099350
0,Marital_StatusOptBin,0.768844
0,SP_F2_33OptBin,0.671356
0,AFF_B_09_01OptBin,0.598777


In [46]:
df_iv.to_csv('IV_optbins_neg_sep_I_bad.csv')

In [47]:
assert df_iv.shape[0] == len(COLUMNS_DICT)
assert not df_iv.IV.isna().any()

In [48]:
df.to_csv(os.path.join(DATA_PATH, 'final_aiq2_optbins_neg_sep_I_bad.csv'))